In [1]:
import torch

In [2]:
#Sea x una entrada de dimensión 512 con distribución N(0,1)
# Media: 0
# Varianza: 1

x = torch.randn(512) # esto es Xo, el resultado es X1 = Xo w1 + b1

#Simular la pasada forward de la entrada con 100 capas lineales
#Las capas tienen 512 neuronas cada una

for i in range(100):
    w = torch.randn(512,512)
    x = w @ x # matrix multiplication dot(a, b)
x.mean(), x.std()

# 100 capas de dim 512, por ende w1 tiene dim 512x512 y b1 tiene dim 512x512, y así hasta w100 y b100

#Media y desviación del resultado explotan -> nan (productos con N(0,1))

(tensor(nan), tensor(nan))

In [3]:
# En qué capa sucede la explosión?

x = torch.randn(512)

for i in range(100):
    a = torch.randn(512,512)
    x = a @ x
    if torch.isnan(x.std()): break

print(i)

#Entrada es pequeña, la única razón para la explosión es que los pesos son muy grandes

27


In [4]:
#Podemos vernos tentados a reducir los pesos para evitar la explosión
# Escalamos los pesos por algún factor

x = torch.randn(512)

for i in range(100):
    a = torch.randn(512,512) * 0.01
    x = a @ x
x.mean(), x.std()

# Ahora la media y la desviación se fueron a cero

(tensor(0.), tensor(0.))

In [5]:
# ¿Cuál es el promedio y desviación estándar de multiplicar un vector de 512 dimensiones y una matriz 512x512?
# Ambos en N(0,1)

#Ejecutamos 10000 multiplicaciones, y promediamos los resultados

import math

mean, var = 0.,0.
for i in range(10000):
    x = torch.randn(512)
    a = torch.randn(512,512)
    y = a @ x
    mean += y.mean().item()
    var += y.pow(2).mean().item()
mean/10000, math.sqrt(var/10000)

(-0.013727493983134628, 22.643891870461175)

In [6]:
#La desviación es muy similar a la raiz cuadrada de la dimension del vector de entrada

math.sqrt(512) # la desviacion es proporcional a la raiz cuadrada de la dimension del vector de entrada

22.627416997969522

In [7]:
#El producto de dos números en distribucipón N(0,1) es siempre un número en la misma distribución

mean, var = 0.,0.
for i in range(10000):
    x = torch.randn(1)
    a = torch.randn(1)
    y = a*x
    mean += y.item()
    var += y.pow(2).item()
mean/10000, math.sqrt(var/10000)


(-0.013396202174478936, 1.0040449287803521)

In [8]:
#La varianza promedio debe estar en el orden de 1/512
mean, var = 0.,0.
for i in range(10000):
    x = torch.randn(1)
    a = torch.randn(1)*math.sqrt(1./512)
    y = a*x
    mean += y.item()
    var += y.pow(2).item()
mean/10000, var/10000

(-0.00015903993283615138, 0.0019520228149370736)

In [9]:
1/512

0.001953125

In [10]:
#Así que deberíamos usar sqrt(1/512) para escalar los pesos

mean, var = 0.,0.
for i in range(10000):
    x = torch.randn(512)
    a = torch.randn(512,512) * math.sqrt(1./512) # antes era 0.01
    y = a @ x
    mean += y.mean().item()
    var += y.pow(2).mean().item()
mean/10000, math.sqrt(var/10000)  # se mantienen acotado, ya no son nan o 0.0

(-0.0004002824828145094, 1.0005287594472172)

In [11]:
#Probemos en nuestra red neuronal simulada

x = torch.randn(512)

for i in range(100):
    a = torch.randn(512,512) * math.sqrt(1./512)
    x = a @ x
x.mean(), x.std()

#Las salidas no explotan ni se desvanecen

(tensor(0.0508), tensor(0.5467))

In [12]:
#Hasta ahora no hemos utilizado funciones de activación.
#Veamos que pasa si aplicamos una función de activación
#TANH a nuestro modelo basico de red neuronal

def tanh(x): return torch.tanh(x)

In [13]:
x = torch.randn(512)

for i in range(100):
    a = torch.randn(512,512) * math.sqrt(1./512)
    x = tanh(a @ x)
x.mean(), x.std()

(tensor(0.0004), tensor(0.0535))

In [14]:
x = torch.randn(512)

for i in range(100):
    a = torch.Tensor(512,512).uniform_(-1, 1) * math.sqrt(1./512)
    x = tanh(a @ x)
x.mean(), x.std()

(tensor(2.1927e-26), tensor(9.6142e-25))

In [15]:
#Glorot y Bengio propusieron una nueva inicialización
def xavier(m,h):
    return torch.Tensor(m, h).uniform_(-1, 1)*math.sqrt(6./(m+h))

x = torch.randn(512)

for i in range(100):
    a = xavier(512, 512)
    x = tanh(a @ x)
x.mean(), x.std()

(tensor(0.0018), tensor(0.0651))

In [16]:
#Pero que pasa cuando la función de activación es RELU?
def relu(x): return x.clamp_min(0.)

In [17]:
mean, var = 0.,0.
for i in range(10000):
    x = torch.randn(512)
    a = torch.randn(512,512)
    y = relu(a @ x)
    mean += y.mean().item()
    var += y.pow(2).mean().item()
mean/10000, math.sqrt(var/10000)

(9.040104362583161, 16.02591488882887)

In [18]:
#Desviación estándar es cernaca a sqrt(512)/sqrt(2)
math.sqrt(512/2)

16.0

In [19]:
mean, var = 0.,0.
for i in range(10000):
    x = torch.randn(512)
    a = torch.randn(512,512)*math.sqrt(2/512)
    y = relu(a @ x)
    mean += y.mean().item()
    var += y.pow(2).mean().item()
mean/10000, math.sqrt(var/10000)

(0.564910521799326, 1.0013181406520248)

In [20]:
def kaiming(m,h):
  return torch.randn(m,h)*math.sqrt(2./m)

x = torch.randn(512)

for i in range(100):
  a = kaiming(512, 512)
  x = relu(a @ x)

x.mean(), x.std()

(tensor(0.3962), tensor(0.5374))

In [21]:
#Xavier con RELU?

x = torch.randn(512)

for i in range(100):
  a = xavier(512, 512)
  x = relu(a @ x)

x.mean(), x.std()

(tensor(3.4718e-16), tensor(4.9262e-16))

In [22]:
!python -V

Python 3.10.12


In [32]:
!date

Thu Aug 31 07:08:42 PM UTC 2023
